In [1]:
import numpy as np
import pandas as pd

In [2]:
dfx = pd.read_csv('Trainoddoneout.csv')
y_train = dfx["survived"]
#print(y_train)
dfy = pd.read_csv('Testtitanic.csv')
dfx.sample()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
428,3.0,0.0,"Vander Planke, Miss. Augusta Maria",female,18.0,2.0,0.0,345764,18.0,NaN,S,NaN,NaN,NaN


In [3]:
columns_to_drop = ["name", "ticket", "fare", "cabin", "embarked", "home.dest", "boat", "body"]
data_clean = dfx.drop(columns_to_drop, axis = 1)
data_clean.head()

,pclass,survived,sex,age,sibsp,parch
0,3.0,0.0,female,NaN,0.0,0.0
1,2.0,0.0,male,39.0,0.0,0.0
2,2.0,1.0,female,40.0,0.0,0.0
3,3.0,1.0,female,31.0,1.0,1.0
4,3.0,1.0,female,NaN,2.0,0.0


In [27]:
data = dfy.drop(columns_to_drop, axis = 1)
data.head()

,pclass,sex,age,sibsp,parch
0,1.0,male,36.0,0.0,0.0
1,3.0,female,NaN,8.0,2.0
2,1.0,male,NaN,0.0,0.0
3,2.0,male,34.0,0.0,0.0
4,2.0,male,28.0,0.0,0.0


In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data_clean["sex"] = le.fit_transform (data_clean["sex"])

data_clean.sample()

,pclass,survived,sex,age,sibsp,parch
537,2.0,0.0,1,25.0,1.0,2.0


In [30]:
data["sex"] = le.fit_transform (data["sex"])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
pclass    300 non-null float64
sex       300 non-null int64
age       234 non-null float64
sibsp     300 non-null float64
parch     300 non-null float64
dtypes: float64(4), int64(1)
memory usage: 11.8 KB


In [5]:
data_cleaned = data_clean.fillna(data_clean["age"].mean())
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 6 columns):
pclass      1009 non-null float64
survived    1009 non-null float64
sex         1009 non-null int64
age         1009 non-null float64
sibsp       1009 non-null float64
parch       1009 non-null float64
dtypes: float64(5), int64(1)
memory usage: 47.4 KB


In [31]:
data_cleed = data.fillna(data["age"].mean())
data_cleed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
pclass    300 non-null float64
sex       300 non-null int64
age       300 non-null float64
sibsp     300 non-null float64
parch     300 non-null float64
dtypes: float64(4), int64(1)
memory usage: 11.8 KB


In [32]:
input_cols = ["pclass", "sex", "age", "sibsp", "parch"]
output_cols = ["survived"]
x = data_cleaned[input_cols]
y = data_cleaned[output_cols]
z = data_cleed[input_cols]
print(z.shape)
print(x.shape)

(300, 5)
(1009, 5)


In [16]:
def entropy(col):
    count = np.unique(col, return_counts = True)
    n = float(col.shape[0])
    ent = 0.0
    for i in count[1]:
        p = i/n
        ent += (-1.0*p*np.log2(p))
    return ent


In [17]:
print(entropy(x["age"]))

5.446006743166156


In [9]:
def divide_data(x_data, fkey, fval):
    x_right = pd.DataFrame([], columns = x_data.columns)
    x_left = pd.DataFrame([], columns = x_data.columns)
    for i in range(x_data.shape[0]):
        val = x_data[fkey].loc[i]
        if val > fval:
            x_right = x_right.append(x_data.loc[i])
        else:
            x_left = x_left.append(x_data.loc[i])
    return x_left, x_right
    

In [20]:
x_left, x_right = divide_data(data_clean[:10], "sex", 0.5)
print(x_left)
print(x_right)

   pclass  survived  sex   age  sibsp  parch
0     3.0       0.0  0.0   NaN    0.0    0.0
2     2.0       1.0  0.0  40.0    0.0    0.0
3     3.0       1.0  0.0  31.0    1.0    1.0
4     3.0       1.0  0.0   NaN    2.0    0.0
7     1.0       1.0  0.0  49.0    0.0    0.0
   pclass  survived  sex   age  sibsp  parch
1     2.0       0.0  1.0  39.0    0.0    0.0
5     2.0       0.0  1.0  16.0    0.0    0.0
6     2.0       0.0  1.0  43.0    0.0    1.0
8     1.0       0.0  1.0  48.0    0.0    0.0
9     2.0       0.0  1.0   NaN    0.0    0.0


In [21]:
def info_gain(x_data, fkey, fval):
    left, right = divide_data(x_data, fkey, fval)
    l = float(left.shape[0]/x_data.shape[0])
    r = float(right.shape[0]/x_data.shape[0])
    if left.shape[0] == 0 or right.shape[0] == 0:
        return -10000000 #min info gain
    i_gain = entropy(x_data.survived) - (l*entropy(left.survived) + r*entropy(right.survived)) 
    return i_gain


In [25]:
for fx in x.columns:
    print(fx)
    print(info_gain(data_cleaned, fx, data_cleaned[fx].mean()))


pclass
0.055456910002982474
sex
0.19274737190850932
age
0.0010525742338489685
sibsp
0.006492394392888956
parch
0.01975608012294816


In [26]:
class DecisionTree:
    def __init__(self, depth = 0, max_depth = 5):
        self.left = None
        self.right = None
        self.fkey = None
        self.fval = None
        self.depth = depth
        self.max_depth = max_depth
        self.target = None
    def train(self, x_train):
        features = ["pclass", "sex", "age", "sibsp", "parch"]
        info_gains = []
        for fx in features:
            i_gain = info_gain(x_train, fx, x_train[fx].mean())
            info_gains.append(i_gain)
        self.fkey = features[np.argmax(info_gains)]
        self.val = x_train[self.fkey].mean()
        print("makin tree feature is",self.fkey)
        data_left, data_right = divide_data(x_train, self.fkey, self.fval)
        data_left = data_left.reset_index(drop = True)
        data_right = data_right.reset_index(drop = True)
        #truly a leaf node
        if data_left.shape[0] == 0 or data_right.shape[0] == 0:
            if x_train.survived.mean() >= 0.5:
                self.target = "survive"
            else:
                self.target = "dead"
            return
        #stop early whendepth >= max depth
        if(self.depth >= self.max_depth):
            if x_train.survived.mean() >= 0.5:
                self.target = "survive"
            else:
                self.target = "dead"
            return
        #recursive case
        self.left = DecisionTree(depth = self.depth + 1, max_depth = self.max_depth)
        self.left.train(data_left)
        self.right = DecisionTree(depth = self.depth + 1, max_depth = self.max_depth)
        self.right.train(data_right)
        #you can set atrget at every node
        if x_train.survived.mean() >= 0.5:
            if x_train.survived.mean() >= 0.5:
                self.target = "survive"
            else:
                self.target = "dead"

In [33]:
from sklearn.tree import DecisionTreeClassifier

In [34]:
sk = DecisionTreeClassifier(criterion = 'entropy', max_depth = 5)

In [35]:
sk.fit(x, y)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [38]:
arr = sk.predict(z)
print(arr)

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0.
 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [40]:
df = pd.DataFrame(arr)
df.columns = ['survived']
df.to_csv('submission_titanic.csv', encoding = 'utf-8')